In this notebook we will use the same Cython code as in the last notebook. However, this time we will use the `Vode` integrator from `ODEPACK` (available in SciPy in `scipy.integrate.ode`). The reason for this is that it will be a fairer comparison against our upcoming example using `CVode`.

In [ ]:
import json
import numpy as np
from odesys import ODEsys
from chem import odesys_from_reactions_names_and_params

Subclassing `ODEsys` and providing a new method using `scipy.integrate.ode`:

In [ ]:
# %load odesys_vode.py
from odesys import ODEsys

from scipy.integrate import ode
from odesys import ODEsys

class VODEsys(ODEsys):
    def integrate_vode(self, tout, y0, params=(), method='bdf', **kwargs):
        r = ode(self.f_eval, jac=self.j_eval)
        r.set_integrator('vode', method=method, **kwargs)
        if params:
            r.set_f_params(params)
            r.set_jac_params(params)
        r.set_initial_value(y0, tout[0])
        yout = np.zeros((len(tout), len(y0)))
        yout[0, :] = y0
        for idx in range(1, len(tout)):
            r.integrate(tout[idx])
            assert r.successful(), "Integration failed"
            yout[idx, :] = r.y
        return yout, {
            'num_steps': r.iwork[10], 'num_rhs': r.iwork[11], 'num_jac': r.iwork[12],
            'num_lu_factor': r.iwork[18], 'num_nonlin_solv_iters': r.iwork[19],
            'num_nonlin_solv_conv_failures': r.iwork[20], 'num_err_test_fails': r.iwork[21]
        }


Creating a new mixin class:

In [ ]:
from cython_ode import CythonODEsys

class CythonVODEsys(VODEsys, CythonODEsys):
    pass

Same procuedure as in the last notebook:

In [ ]:
watrad_data = json.load(open('radiolysis_300_Gy_s.json'))
watrad = odesys_from_reactions_names_and_params(ODEsys, **watrad_data)
tout = np.logspace(-6, 3, 200)  # close to one hour of operation
c0 = {'H2O': 55.4e3, 'H+': 1e-4, 'OH-': 1e-4}
y0 = [c0.get(symb.name, 0) for symb in watrad.y]

In [ ]:
cython_sys = odesys_from_reactions_names_and_params(CythonVODEsys, **watrad_data)

In [ ]:
%timeit cython_sys.integrate_vode(tout, y0)

In [ ]:
%debug

That is a considerable speed up from before. But the solver still has to
allocate memory for creating new arrays at each call, and each evaluation
has to pass the python layer which is now the bottleneck for the integration.

In order to speed up integration further we need to make sure the solver can evaluate the function and jacobian without calling into Python.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Just to see that everything looks alright:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
cython_sys.plot_result(tout, *cython_sys.integrate_odeint(tout, y0), ax=ax)
ax.set_xscale('log')
ax.set_yscale('log')